---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [5]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [6]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    file = pd.read_csv('university_towns.txt', sep='\n', usecols=[0],header=None, names=['raw']\
                       ,index_col=False)
    file['State']='None'; file['RegionName'] = 'None'
    # iterate over every rows
    for index, row in file.iterrows():
        # find the state name
        if row['raw'].find('[edit]') != -1:
            idx = row['raw'].find('[edit]')
            state = row['raw'][:idx]
        # if not state, extract the region name by spliting with parenthesis
        else:
            regionname = row['raw'].split('(')[0].strip()
            row['RegionName'] = regionname
            row['State'] = state

    uni_town = file.loc[file['State'] != 'None',['State','RegionName']].reset_index(drop=True)
    return uni_town

In [7]:
# READ IN GDP DATA
# process it for later use
gdp = pd.read_excel('gdplev.xls', skiprows=range(0,7), usecols=[4,6], names=['Quarter','ChainedGDP'])

gdp_inuse = gdp[gdp.Quarter >= '2000q1'].reset_index(drop=True)

gdp_inuse['Change'] = gdp_inuse.ChainedGDP.diff()

In [241]:
#gdp_inuse[gdp_inuse['Quarter']>'2007']

In [8]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    #  recession的开始定义为chained gdp连续两个季度下降。第一个看到下降的时间点就是recession_start
    #  period1为第一个下降的quarter, period2为第二个下降的quarter
    #  通过不断更新period1和period2，并计算他们之间的“距离”，来寻找连续下降的时间点。
    period1 = None
    recession_start=0
    for index, row in gdp_inuse.iterrows():    
        if row['Change'] <0:
            period2 = index
            if period1 != None:
                if (period2 - period1) == 1: # 两个下降期是否紧邻？
                    recession_start = period1 
                    break  #只要找到recession_start，马上跳出循环
                else: period1 = period2
            else: period1 = period2
    return gdp_inuse['Quarter'][recession_start]

In [9]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    start = get_recession_start()
    recession = gdp_inuse[gdp_inuse.Quarter>=start]
    period1=None
    recession_end=0
    for index, row in recession.iterrows():    
        if row['Change'] >0:
            period2 = index
            if period1 != None:
                if (period2 - period1) == 1: # 两个上升期是否紧邻？
                    recession_end = period2 
                    break  #只要找到recession_start，马上跳出循环
                else: period1 = period2
            else: period1 = period2    
    return gdp_inuse['Quarter'][recession_end]

In [242]:
#get_recession_end()

In [10]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    start = get_recession_start()
    end = get_recession_end()
    recession = gdp_inuse[gdp_inuse.Quarter.between(start,end)].set_index('Quarter')
    bottom = recession.ChainedGDP.idxmin()
    return bottom

In [243]:
#get_recession_bottom()

In [207]:
# bad, inefficient way. abandoned
# housing = pd.read_csv('City_Zhvi_AllHomes.csv')

# housing.set_index(['State','RegionName'],inplace=True)

# housing = housing.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'],axis=1)
# # make a mapping table which converts '01' to 'q1'
# quarter={} 
# for i in range(1,13):
#     if len(str(i)) !=2:
#         month_str = '0'+str(i)
#     else: month_str=str(i)
#     if i in range(1,4):
#         quarter[month_str] = 'q1'
#     if i in range(4,7):
#         quarter[month_str] = 'q2'
#     if i in range(7,10):
#         quarter[month_str] = 'q3'
#     if i in range(10,13): 
#         quarter[month_str] = 'q4'
# # stack all columns into a new column called 'month'. The value looks like '1996-01','1997-08'
# # name the value of each year's house price as 'price'
# housing_stacked = housing.stack(dropna=False).reset_index(level=2,name='price')
# housing_stacked.rename(columns={'level_2':'month'},inplace=True)

# # create a new column that transform 'month' to quarterly representation
# f = lambda m: m.split('-')[0]+quarter[m.split('-')[1]]
# housing_stacked['quarter'] = housing_stacked['month'].apply(f)

In [20]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing['State'] = housing['State'].apply(lambda x: states[x])
    housing.set_index(['State','RegionName'],inplace=True)
    newcol = []
    for col in housing.columns:
        if col >='2000-01' and col <= '2016-08':
            newcol.append(col)
    housing = housing[newcol]
    housing = housing.rename(columns=pd.to_datetime) # column can also be DatetimeIndex type!
    housing = housing.resample('Q',axis=1).mean()
    housing.rename(columns=lambda x: str(x.to_period('Q')).lower(), inplace=True)
    return housing

In [83]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    housing = convert_housing_data_to_quarters()
    start = get_recession_start()
    bottom = get_recession_bottom()
    ut_list = get_list_of_university_towns()

    #find the quarter before recession started.
    for idx, col in enumerate(housing.columns):
        if col == start:
            qtr_before_idx = idx -1

    qtr_before_rec_start = housing.columns[qtr_before_idx]
    sel_col = [col for col in housing.columns if col >=qtr_before_rec_start and col <= bottom]

    testdata = pd.merge(left=housing[sel_col], right=ut_list, how='left', left_index=True,\
             right_on=['State','RegionName'],indicator=True)
    testdata['is_univeristy_town'] = testdata['_merge'].apply(lambda x: '0' if x=='left_only' else '1')
    testdata = testdata.drop(['_merge'],axis=1).reset_index(drop=True)

    # compute price ratio using the formula given above
    testdata['price_ratio'] = testdata[qtr_before_rec_start] / testdata[bottom]

    # split the test data
    ut = testdata[testdata['is_univeristy_town'] == '1']
    non_ut = testdata[testdata['is_univeristy_town'] == '0']

    t_stats, p = ttest_ind(ut['price_ratio'],non_ut['price_ratio'],nan_policy='omit')

    different = True if p < 0.01 else False

    better = 'university town' if ut['price_ratio'].mean() < non_ut['price_ratio'].mean() else 'non-university town'
    return (different, p, better)

In [84]:
run_ttest()

(True, 0.0027240637047531249, 'university town')